### AccuRator 서술형 검색

#### Step.1 AccuRator 서비스 화면의 입력Text(해결하고자 하는 문제나 과제내용)를 question 변수에 대입

In [14]:
# 화면을 통한 사용자 질의내용 Sample1
question = "당사는 식료품 제조업을영위하고 있습니다. 팬데믹으로 인해 소비의 형태가 많이 바뀌었고, 매출에 영향을 주고 있습니다.이에 코로나로 인한 소비형태 변화를 반영해 수요예측을 하고 싶습니다."


In [15]:
# 화면을 통한 사용자 질의내용 Sample 2
question = "프렌차이즈 본사입니다.주요 교통량, 유동인구, 지역별 자산수준을 고려하여, 지점의 입지후보 모델을 개발하고 싶습니다.모형개발하는데 검토할 수 있는 데이터들을 찾아주세요."


#### Step2.질의내용(question)에 대한 NLP Process

In [16]:
from konlpy.tag import Okt, Mecab
import re

def get_tokenizer(tokenizer_name):
    if tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab(dicpath="C:/mecab/mecab-ko-dic")
    else:
        tokenizer = Mecab(dicpath="C:/mecab/mecab-ko-dic")
    return tokenizer

In [17]:
question = question.rstrip('\n')
question = question.strip()
question = question.replace(",","")
question = question.replace("."," ")
question = re.sub('[^가-힣a-z]', ' ', question)   
print(question)

프렌차이즈 본사입니다 주요 교통량 유동인구 지역별 자산수준을 고려하여 지점의 입지후보 모델을 개발하고 싶습니다 모형개발하는데 검토할 수 있는 데이터들을 찾아주세요 


In [18]:
tokenizer = get_tokenizer("mecab")
tokenizer.pos(question)
tokenizer.nouns(question)

tokens=[]
tokens = tokenizer.nouns(question)
print(tokens)
print(len(tokens))

['프렌차이즈', '본사', '주요', '교통량', '유동', '인구', '지역', '자산', '수준', '고려', '지점', '입지', '후보', '모델', '개발', '모형', '개발', '데', '검토', '수', '데이터']
21


In [23]:
for s in tokens[:]:
    if len(s) < 2:
        tokens.remove(s)

In [24]:
def stopwords_processing(token,tokenizer):
    
    stopwords = [
 '영위'
,'당사'
,'형태'
,'변화'
,'반영'
,'예측'
,'본사'
,'주요'
,'수준'
,'고려'
,'후보'
,'모델'
,'모형'
,'개발'
,'검토'
,'데이터']  
    token = [t for t in token if t not in stopwords]       
    return token

tokens= stopwords_processing(tokens,tokenizer)
print(tokens)
print(len(tokens))

['프렌차이즈', '교통량', '유동', '인구', '지역', '자산', '지점', '입지']
8


#### Step3. NLP process를 통한 "키워드 리스트"를 활용하여 SQL 동적작성 및 실행

고려사항 <br>

(1) 하기 3개의 Case를 조회하면 중복 데이터가 발생할 것.
이를 한번 더 싸서 중복카운트가 높은 데이터순으로 화면에 출력해 주는 방식도 고려하면 좋음
<br>
(2) Limit 고려 (최대 200 개)
<br>
(3) 화면에 출력시 현재 화면의 "결합속성명"은 제외할 것, 맥락에 안맞음
<br>
(4) DB에 따라 하기와 같이 regexp_like 문 활용하면 SQL 동적작성이 편함
WHERE regexp_like(name, 'AA|BB|DD')

In [ ]:
## SQL 예시

-- Case1 : 데이터명에 like 검색
select 산업, 데이터명, 제공처, 갱진주기, 유료/무료 ,가격
  from accurator_master
 where 1=1
   and ( data_nm like '%프랜차이즈%' or  
         data_nm like '%교통량%' or
         data_nm like '%유동%' or
         data_nm like '%인구%' or
         data_nm like '%지역%' or
         data_nm like '%자산%' or
         data_nm like '%지점%' or
         data_nm like '%입지%' )
         
union all
-- case2 : Curation Thema명에 like검색
select 산업, 데이터명, 제공처, 갱진주기, 유료/무료 ,가격
  from accurator_curation_master
 where 1=1
   and ( 큐레아션추천소분류  like '%프랜차이즈%' or  
         큐레아션추천소분류 like '%교통량%' or
         큐레아션추천소분류 like '%유동%' or
         큐레아션추천소분류 like '%인구%' or
         큐레아션추천소분류 like '%지역%' or
         큐레아션추천소분류 like '%자산%' or
         큐레아션추천소분류 like '%지점%' or
         큐레아션추천소분류 like '%입지%' )
 
 union all 
-- case3 : 결합키에 like검색
select 산업, 데이터명, 제공처, 갱진주기, 유료/무료 ,가격
  from accurator_master
 where 1=1
   and ( 결합키  like '%프랜차이즈%' or  
         결합키  like '%교통량%' or
         결합키  like '%유동%' or
         결합키  like '%인구%' or
         결합키  like '%지역%' or
         결합키  like '%자산%' or
         결합키  like '%지점%' or
         결힙키  like '%입지%' )